# Convert a Keras model to TensorFlow.js format

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
import tensorflowjs as tfjs

# Define your model architecture here
model = Sequential()
model.add(Input(shape=(30,1662)))
model.add(LSTM(64, return_sequences=True, activation='relu', name='lstm1'))
model.add(Dense(64, activation='relu', name='dense1'))
model.add(LSTM(128, return_sequences=True, activation='relu', name='lstm2'))
model.add(Dense(64, activation='relu', name='dense2'))
model.add(LSTM(64, return_sequences=False, activation='relu', name='lstm3'))
model.add(Dense(32, activation='relu', name='dense3'))
model.add(Dense(3, activation='softmax', name='output'))  # replace 3 with the number of your classes

# Load the weights into the model
model.load_weights('melav01_renamed.h5')

# Save the model in TensorFlow.js format
tfjs.converters.save_keras_model(model, './output_directory')

failed to lookup keras version from the file,
    this is likely a weight only file


Exception: Error dumping weights, duplicate weight name kernel